In [1]:
import numpy as np
import pandas as pd
from sklearn import ensemble, neighbors, linear_model, metrics, preprocessing
from datetime import datetime

In [26]:
data = {
    'tra': pd.read_csv('/content/drive/MyDrive/aiffel/Hackerton1/air_visit_data.csv'),
    'as': pd.read_csv('/content/drive/MyDrive/aiffel/Hackerton1/air_store_info.csv'),
    'hs': pd.read_csv('/content/drive/MyDrive/aiffel/Hackerton1/hpg_store_info.csv'),
    'ar': pd.read_csv('/content/drive/MyDrive/aiffel/Hackerton1/air_reserve.csv'),
    'hr': pd.read_csv('/content/drive/MyDrive/aiffel/Hackerton1/hpg_reserve.csv'),
    'id': pd.read_csv('/content/drive/MyDrive/aiffel/Hackerton1/store_id_relation.csv'),
    'tes': pd.read_csv('/content/drive/MyDrive/aiffel/Hackerton1/sample_submission.csv'),
    'hol': pd.read_csv('/content/drive/MyDrive/aiffel/Hackerton1/date_info.csv').rename(columns={'calendar_date':'visit_date'})
    }


In [27]:
data['hr'] = pd.merge(data['hr'], data['id'], how='inner', on=['hpg_store_id'])
  # hpg_reserve.csv에 store_id_relation.csv를 hpg_store_id 기준으로 inner merge

data['hr'].head()

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors,air_store_id
0,hpg_878cc70b1abc76f7,2016-01-01 19:00:00,2016-01-01 15:00:00,4.0,air_db80363d35f10926
1,hpg_878cc70b1abc76f7,2016-01-02 19:00:00,2016-01-02 14:00:00,2.0,air_db80363d35f10926
2,hpg_878cc70b1abc76f7,2016-01-03 18:00:00,2016-01-02 20:00:00,6.0,air_db80363d35f10926
3,hpg_878cc70b1abc76f7,2016-01-06 20:00:00,2016-01-04 22:00:00,3.0,air_db80363d35f10926
4,hpg_878cc70b1abc76f7,2016-01-11 18:00:00,2016-01-11 14:00:00,2.0,air_db80363d35f10926


In [28]:
for df in ['ar','hr']:
    data[df]['visit_datetime'] = pd.to_datetime(data[df]['visit_datetime'])
      # str을 datetime으로 변환
    data[df]['visit_datetime'] = data[df]['visit_datetime'].dt.date
      # datetime을 dt.date로 변환 
    data[df]['reserve_datetime'] = pd.to_datetime(data[df]['reserve_datetime'])
      # str을 datetime으로 변환
    data[df]['reserve_datetime'] = data[df]['reserve_datetime'].dt.date
      # datetime을 dt.date로 변환 
    data[df]['reserve_datetime_diff'] = data[df].apply(lambda r: (r['visit_datetime'] - r['reserve_datetime']).days, axis=1)
      # difference time을 의미하는 변수 생성 
    data[df] = data[df].groupby(['air_store_id','visit_datetime'], as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].sum().rename(columns={'visit_datetime':'visit_date'})
      # air_reserve와 hpg_ reserve로 group을 나눔
    print(data[df].head())

           air_store_id  visit_date  reserve_datetime_diff  reserve_visitors
0  air_00a91d42b08b08d9  2016-10-31                      0                 2
1  air_00a91d42b08b08d9  2016-12-05                      4                 9
2  air_00a91d42b08b08d9  2016-12-14                      6                18
3  air_00a91d42b08b08d9  2016-12-17                      6                 2
4  air_00a91d42b08b08d9  2016-12-20                      2                 4
           air_store_id  visit_date  reserve_datetime_diff  reserve_visitors
0  air_00a91d42b08b08d9  2016-01-14                      3               2.0
1  air_00a91d42b08b08d9  2016-01-15                      6               4.0
2  air_00a91d42b08b08d9  2016-01-16                      3               2.0
3  air_00a91d42b08b08d9  2016-01-22                      3               2.0
4  air_00a91d42b08b08d9  2016-01-29                      6               5.0


In [29]:
# air_visit_data

data['tra']['visit_date'] = pd.to_datetime(data['tra']['visit_date'])
  # str을 datetime으로 변환
data['tra']['dow'] = data['tra']['visit_date'].dt.dayofweek
  # 요일을 의미하는 숫자 변수 생성
data['tra']['year'] = data['tra']['visit_date'].dt.year
  # 년도 year 변수 생성
data['tra']['month'] = data['tra']['visit_date'].dt.month
  # 달 month 변수 생성
data['tra']['visit_date'] = data['tra']['visit_date'].dt.date
  # datetime을 dt.date로 변환
data['tra'].head()

,air_store_id,visit_date,visitors,dow,year,month
0,air_ba937bf13d40fb24,2016-01-13,25,2,2016,1
1,air_ba937bf13d40fb24,2016-01-14,32,3,2016,1
2,air_ba937bf13d40fb24,2016-01-15,29,4,2016,1
3,air_ba937bf13d40fb24,2016-01-16,22,5,2016,1
4,air_ba937bf13d40fb24,2016-01-18,6,0,2016,1


In [30]:
# sample_submission

data['tes']['visit_date'] = data['tes']['id'].map(lambda x: str(x).split('_')[2])
  # id 변수에서 날짜를 뽑아와 visit_date 변수 생성
data['tes']['air_store_id'] = data['tes']['id'].map(lambda x: '_'.join(x.split('_')[:2]))
  # id 변수에서 store id를 뽑아와 air_store_id 변수 생성
data['tes']['visit_date'] = pd.to_datetime(data['tes']['visit_date'])
  # str을 datetime으로 변환
data['tes']['dow'] = data['tes']['visit_date'].dt.dayofweek
  # 요일을 의미하는 숫자 변수 생성
data['tes']['year'] = data['tes']['visit_date'].dt.year
  # 년도 year 변수 생성
data['tes']['month'] = data['tes']['visit_date'].dt.month
  # 달 month 변수 생성
data['tes']['visit_date'] = data['tes']['visit_date'].dt.date
  # datetime을 dt.date로 변환

data['tes'].head()

,id,visitors,visit_date,air_store_id,dow,year,month
0,air_00a91d42b08b08d9_2017-04-23,0,2017-04-23,air_00a91d42b08b08d9,6,2017,4
1,air_00a91d42b08b08d9_2017-04-24,0,2017-04-24,air_00a91d42b08b08d9,0,2017,4
2,air_00a91d42b08b08d9_2017-04-25,0,2017-04-25,air_00a91d42b08b08d9,1,2017,4
3,air_00a91d42b08b08d9_2017-04-26,0,2017-04-26,air_00a91d42b08b08d9,2,2017,4
4,air_00a91d42b08b08d9_2017-04-27,0,2017-04-27,air_00a91d42b08b08d9,3,2017,4


In [31]:
# ??????
unique_stores = data['tes']['air_store_id'].unique()
stores = pd.concat([pd.DataFrame({'air_store_id': unique_stores, 
                                  'dow': [i]*len(unique_stores)}) for i in range(7)], 
                   axis=0, 
                   ignore_index=True).reset_index(drop=True)

stores.head()

,air_store_id,dow
0,air_00a91d42b08b08d9,0
1,air_0164b9927d20bcc3,0
2,air_0241aa3964b7f861,0
3,air_0328696196e46f18,0
4,air_034a3d5b40d5b1b1,0


In [32]:
# sure it can be compressed...

tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].min().rename(columns={'visitors':'min_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow']) 
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].mean().rename(columns={'visitors':'mean_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].median().rename(columns={'visitors':'median_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].max().rename(columns={'visitors':'max_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].count().rename(columns={'visitors':'count_observations'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow']) 

stores.head()

,air_store_id,dow,min_visitors,mean_visitors,median_visitors,max_visitors,count_observations
0,air_00a91d42b08b08d9,0,1.0,22.457143,19.0,47.0,35.0
1,air_0164b9927d20bcc3,0,2.0,7.500000,6.0,19.0,20.0
2,air_0241aa3964b7f861,0,NaN,NaN,NaN,NaN,NaN
3,air_0328696196e46f18,0,NaN,NaN,NaN,NaN,NaN
4,air_034a3d5b40d5b1b1,0,1.0,11.864865,10.0,66.0,37.0


In [33]:
stores = pd.merge(stores, data['as'], how='left', on=['air_store_id']) 
stores.head()

,air_store_id,dow,min_visitors,mean_visitors,median_visitors,max_visitors,count_observations,air_genre_name,air_area_name,latitude,longitude
0,air_00a91d42b08b08d9,0,1.0,22.457143,19.0,47.0,35.0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595
1,air_0164b9927d20bcc3,0,2.0,7.500000,6.0,19.0,20.0,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599
2,air_0241aa3964b7f861,0,NaN,NaN,NaN,NaN,NaN,Izakaya,Tōkyō-to Taitō-ku Higashiueno,35.712607,139.779996
3,air_0328696196e46f18,0,NaN,NaN,NaN,NaN,NaN,Dining bar,Ōsaka-fu Ōsaka-shi Nakanochō,34.701279,135.528090
4,air_034a3d5b40d5b1b1,0,1.0,11.864865,10.0,66.0,37.0,Cafe/Sweets,Ōsaka-fu Ōsaka-shi Ōhiraki,34.692337,135.472229


In [34]:
lbl = preprocessing.LabelEncoder()
  # labelEncoding 진행 
stores['air_genre_name'] = lbl.fit_transform(stores['air_genre_name'])
stores['air_area_name'] = lbl.fit_transform(stores['air_area_name'])

stores.head()

,air_store_id,dow,min_visitors,mean_visitors,median_visitors,max_visitors,count_observations,air_genre_name,air_area_name,latitude,longitude
0,air_00a91d42b08b08d9,0,1.0,22.457143,19.0,47.0,35.0,6,44,35.694003,139.753595
1,air_0164b9927d20bcc3,0,2.0,7.500000,6.0,19.0,20.0,6,62,35.658068,139.751599
2,air_0241aa3964b7f861,0,NaN,NaN,NaN,NaN,NaN,7,82,35.712607,139.779996
3,air_0328696196e46f18,0,NaN,NaN,NaN,NaN,NaN,4,98,34.701279,135.528090
4,air_034a3d5b40d5b1b1,0,1.0,11.864865,10.0,66.0,37.0,2,102,34.692337,135.472229


In [35]:
data['hol']['visit_date'] = pd.to_datetime(data['hol']['visit_date'])
data['hol']['day_of_week'] = lbl.fit_transform(data['hol']['day_of_week'])
data['hol']['visit_date'] = data['hol']['visit_date'].dt.date
  # holiday의 날짜 변수 가공한뒤 merge
train = pd.merge(data['tra'], data['hol'], how='left', on=['visit_date']) 

train.head()

,air_store_id,visit_date,visitors,dow,year,month,day_of_week,holiday_flg
0,air_ba937bf13d40fb24,2016-01-13,25,2,2016,1,6,0
1,air_ba937bf13d40fb24,2016-01-14,32,3,2016,1,4,0
2,air_ba937bf13d40fb24,2016-01-15,29,4,2016,1,0,0
3,air_ba937bf13d40fb24,2016-01-16,22,5,2016,1,2,0
4,air_ba937bf13d40fb24,2016-01-18,6,0,2016,1,1,0


In [36]:
test = pd.merge(data['tes'], data['hol'], how='left', on=['visit_date'])
test.head()

,id,visitors,visit_date,air_store_id,dow,year,month,day_of_week,holiday_flg
0,air_00a91d42b08b08d9_2017-04-23,0,2017-04-23,air_00a91d42b08b08d9,6,2017,4,3,0
1,air_00a91d42b08b08d9_2017-04-24,0,2017-04-24,air_00a91d42b08b08d9,0,2017,4,1,0
2,air_00a91d42b08b08d9_2017-04-25,0,2017-04-25,air_00a91d42b08b08d9,1,2017,4,5,0
3,air_00a91d42b08b08d9_2017-04-26,0,2017-04-26,air_00a91d42b08b08d9,2,2017,4,6,0
4,air_00a91d42b08b08d9_2017-04-27,0,2017-04-27,air_00a91d42b08b08d9,3,2017,4,4,0


In [37]:
train = pd.merge(data['tra'], stores, how='left', on=['air_store_id','dow']) 
train.head()

,air_store_id,visit_date,visitors,dow,year,month,min_visitors,mean_visitors,median_visitors,max_visitors,count_observations,air_genre_name,air_area_name,latitude,longitude
0,air_ba937bf13d40fb24,2016-01-13,25,2,2016,1,7.0,23.843750,25.0,57.0,64.0,4.0,62.0,35.658068,139.751599
1,air_ba937bf13d40fb24,2016-01-14,32,3,2016,1,2.0,20.292308,21.0,54.0,65.0,4.0,62.0,35.658068,139.751599
2,air_ba937bf13d40fb24,2016-01-15,29,4,2016,1,4.0,34.738462,35.0,61.0,65.0,4.0,62.0,35.658068,139.751599
3,air_ba937bf13d40fb24,2016-01-16,22,5,2016,1,6.0,27.651515,27.0,53.0,66.0,4.0,62.0,35.658068,139.751599
4,air_ba937bf13d40fb24,2016-01-18,6,0,2016,1,2.0,13.754386,12.0,34.0,57.0,4.0,62.0,35.658068,139.751599


In [38]:
test = pd.merge(data['tes'], stores, how='left', on=['air_store_id','dow'])
test.head()

,id,visitors,visit_date,air_store_id,dow,year,month,min_visitors,mean_visitors,median_visitors,max_visitors,count_observations,air_genre_name,air_area_name,latitude,longitude
0,air_00a91d42b08b08d9_2017-04-23,0,2017-04-23,air_00a91d42b08b08d9,6,2017,4,2.0,2.000000,2.0,2.0,1.0,6,44,35.694003,139.753595
1,air_00a91d42b08b08d9_2017-04-24,0,2017-04-24,air_00a91d42b08b08d9,0,2017,4,1.0,22.457143,19.0,47.0,35.0,6,44,35.694003,139.753595
2,air_00a91d42b08b08d9_2017-04-25,0,2017-04-25,air_00a91d42b08b08d9,1,2017,4,1.0,24.350000,24.5,43.0,40.0,6,44,35.694003,139.753595
3,air_00a91d42b08b08d9_2017-04-26,0,2017-04-26,air_00a91d42b08b08d9,2,2017,4,15.0,28.125000,28.0,52.0,40.0,6,44,35.694003,139.753595
4,air_00a91d42b08b08d9_2017-04-27,0,2017-04-27,air_00a91d42b08b08d9,3,2017,4,15.0,29.868421,30.0,47.0,38.0,6,44,35.694003,139.753595


In [41]:
for df in ['ar','hr']:
    train = pd.merge(train, data[df], how='left', on=['air_store_id','visit_date']) 
    test = pd.merge(test, data[df], how='left', on=['air_store_id','visit_date'])

data[df].head()

,air_store_id,visit_date,reserve_datetime_diff,reserve_visitors
0,air_00a91d42b08b08d9,2016-01-14,3,2.0
1,air_00a91d42b08b08d9,2016-01-15,6,4.0
2,air_00a91d42b08b08d9,2016-01-16,3,2.0
3,air_00a91d42b08b08d9,2016-01-22,3,2.0
4,air_00a91d42b08b08d9,2016-01-29,6,5.0


In [42]:
print(train.describe())
print(train.head())

            visitors  ...  reserve_visitors_y
count  211306.000000  ...          422.000000
mean       21.349195  ...            5.665877
std        16.949555  ...            5.547439
min         1.000000  ...            1.000000
25%         9.000000  ...            2.000000
50%        17.000000  ...            4.000000
75%        30.000000  ...            7.000000
max       877.000000  ...           38.000000

[8 rows x 25 columns]
           air_store_id  ... reserve_visitors_y
0  air_ba937bf13d40fb24  ...                NaN
1  air_ba937bf13d40fb24  ...                NaN
2  air_ba937bf13d40fb24  ...                NaN
3  air_ba937bf13d40fb24  ...                NaN
4  air_ba937bf13d40fb24  ...                NaN

[5 rows x 27 columns]


In [43]:
col = [c for c in train if c not in ['id', 'air_store_id','visit_date','visitors']]
train = train.fillna(-1)
test = test.fillna(-1)
# ???????

In [46]:
train.head()

,air_store_id,visit_date,visitors,dow,year,month,min_visitors,mean_visitors,median_visitors,max_visitors,count_observations,air_genre_name,air_area_name,latitude,longitude,reserve_datetime_diff_x,reserve_visitors_x,reserve_datetime_diff_y,reserve_visitors_y,reserve_datetime_diff_x,reserve_visitors_x,reserve_datetime_diff_y,reserve_visitors_y,reserve_datetime_diff_x,reserve_visitors_x,reserve_datetime_diff_y,reserve_visitors_y
0,air_ba937bf13d40fb24,2016-01-13,25,2,2016,1,7.0,23.843750,25.0,57.0,64.0,4.0,62.0,35.658068,139.751599,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,air_ba937bf13d40fb24,2016-01-14,32,3,2016,1,2.0,20.292308,21.0,54.0,65.0,4.0,62.0,35.658068,139.751599,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,air_ba937bf13d40fb24,2016-01-15,29,4,2016,1,4.0,34.738462,35.0,61.0,65.0,4.0,62.0,35.658068,139.751599,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,air_ba937bf13d40fb24,2016-01-16,22,5,2016,1,6.0,27.651515,27.0,53.0,66.0,4.0,62.0,35.658068,139.751599,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,air_ba937bf13d40fb24,2016-01-18,6,0,2016,1,2.0,13.754386,12.0,34.0,57.0,4.0,62.0,35.658068,139.751599,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [44]:
def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred)**0.5

In [47]:
lr = linear_model.LinearRegression(n_jobs=-1)
etc = ensemble.ExtraTreesRegressor(n_estimators=225, max_depth=5, n_jobs=-1, random_state=3)
knn = neighbors.KNeighborsRegressor(n_jobs=-1, n_neighbors=4)

lr.fit(train[col], np.log1p(train['visitors'].values))
etc.fit(train[col], np.log1p(train['visitors'].values))
knn.fit(train[col], np.log1p(train['visitors'].values))

print('RMSE LinearRegressor: ', RMSLE(np.log1p(train['visitors'].values), lr.predict(train[col])))
print('RMSE ExtraTreesRegressor: ', RMSLE(np.log1p(train['visitors'].values), etc.predict(train[col])))
print('RMSE KNNRegressor: ', RMSLE(np.log1p(train['visitors'].values), knn.predict(train[col])))

test['visitors'] = (etc.predict(test[col]) / 2) +(knn.predict(test[col]) / 2)
test['visitors'] = np.expm1(test['visitors']).clip(lower=0.)
test[['id','visitors']].to_csv('lr_submission.csv', index=False, float_format='%.2f')

RMSE LinearRegressor:  0.5570391754533082
RMSE ExtraTreesRegressor:  0.5202026745220975
RMSE KNNRegressor:  0.42598073153062593


visitors 그래프 보고 log 변환 왜했는지 알아보기

역 위치, 날씨 